<a href="https://colab.research.google.com/github/courpiaque/cordef/blob/main/trening_modelu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install label-studio-sdk --quiet
!pip install -U sacremoses --quiet
!pip install -U spacy-transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.8/318.8 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31

In [4]:
from label_studio_sdk import Client
import pandas as pd
import spacy
from spacy.tokens import DocBin
from tqdm.notebook import tqdm
from spacy import util

In [5]:
LABEL_STUDIO_URL = 'https://app.heartex.com/'
API_KEY = 'ac32252c97f25574d668e1169b39c4d352abaaf8'

In [6]:
ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)
ls.check_connection()

{'status': 'UP'}

In [7]:
projects = ls.get_projects()
for project in projects:
  print(f"{project.id} {project.title}")

84465 Podręczniki – Definicje bez Komunikacji społecznej
84464 Podręczniki – Komunikacja społeczna
84330 Teksty naukowe
84329 Filozofuj
84322 Wikipedia
70099 Definicje z Hist. Filozofii W.Tatarkiewicza


In [8]:
project = ls.get_project(70099)

In [9]:
data = project.get_labeled_tasks()

In [10]:
len(data)

518

In [11]:
for item in data:
  text = item['data']['token']

  labels = item['annotations'][0]['result']
  for label in labels:
    label_text = label['value']['text']
    label_start = label['value']['start']
    label_end = label['value']['end']
    label_name = label['value']['labels'][0]

In [12]:
nlp = spacy.blank('pl')
docs = []

for item in data:
  text = item['data']['token']
  doc = nlp.make_doc(text)

  ents = []
  labels = item['annotations'][0]['result']
  for label in labels:
    label_text = label['value']['text']
    label_start = label['value']['start']
    label_end = label['value']['end']
    label_name = label['value']['labels'][0]
    span = doc.char_span(label_start, label_end, label=label_name, alignment_mode="contract")
    if span is None:
      print("Skipping entity")
      print(label_text, label_name)
    else:
      ents.append(span)

  doc.ents = util.filter_spans(ents)
  docs.append(doc)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [13]:
from spacy import displacy
displacy.render(docs[-1], style="ent", jupyter=True)

In [14]:
def convert_data(DATA, output="./train.spacy"):
  nlp = spacy.blank('pl')
  db = DocBin()

  for item in DATA:
    text = item['data']['token']
    doc = nlp.make_doc(text)

    ents = []
    labels = item['annotations'][0]['result']
    for label in labels:
      label_text = label['value']['text']
      label_start = label['value']['start']
      label_end = label['value']['end']
      label_name = label['value']['labels'][0]
      span = doc.char_span(label_start, label_end, label=label_name, alignment_mode="contract")
      if span is None:
        print("Skipping entity")
        print(label_text, label_name)
      else:
        ents.append(span)
    doc.ents = util.filter_spans(ents)
    db.add(doc)
  db.to_disk(output)

In [15]:
def train_test_split(DATA, test):
    train_end = int(len(DATA) * (100-test)*0.01)
    print(train_end)
    test_start = int(len(DATA) * (100-test))+1
    return DATA[0:train_end], DATA[train_end+1:len(DATA)]

In [16]:
train_data, test_data = train_test_split(data, test=30)
test_data, dev_data = train_test_split(test_data, test=50)

362
77


In [17]:
len(dev_data)

77

In [18]:
convert_data(train_data, "./train.spacy")
convert_data(test_data, "./test.spacy")
convert_data(dev_data, "./dev.spacy")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [19]:
import locale
print(locale.getpreferredencoding())
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

UTF-8


In [20]:
!python -m spacy init config --lang pl --pipeline ner config.cfg --force --gpu --optimize efficiency

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
ℹ Generated config template specific for your use case
- Language: pl
- Pipeline: ner
- Optimize for: efficiency
- Hardware: GPU
- Transformer: dkleczek/bert-base-polish-cased-v1
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --paths.train ./train.spacy  --paths.dev ./dev.spacy --output ner_model --gpu-id 0

✔ Created output directory: ner_model
ℹ Saving to output directory: ner_model
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
tokenizer_config.json: 100% 30.0/30.0 [00:

In [ ]:
!python -m spacy evaluate ner_model/model-best/ --output metrics.json ./test.spacy --gpu-id 0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp -r ner_model/model-best/ "/content/drive/MyDrive/model-best"